# **UWB IPS Heuristic Algorithm Experiments**

## **0. Problem Statement**

Given anchors position $\mathbf{A}_0, \mathbf{A}_1, ..., \mathbf{A}_n$ and anchors range reading $r_0, r_1, ..., r_n$, we want to determine the best tag position $\mathbf{P}$, subject to the following objective:

$$\min f(\mathbf{\mathbf{P}}) = \sum_{0 \leq i \leq n} e_i^2$$

where

$$e_i = \lVert\mathbf{A}_i - \mathbf{P}\rVert - r_i$$

Before we proceed to the demonstration, let's import some of the necessary assets.

In [107]:
import numpy as np
import pandas as pd
import yaml

# import all dataset files
DATASET_ID = '3ea8cd5a'
DATA_ACTUAL_PATH = f'./data/{DATASET_ID}/actual.csv'
DATA_READ_PATH = f'./data/{DATASET_ID}/read.csv'
CONFIG_PATH = f'./data/{DATASET_ID}/generate.yaml'
GENERATE_YAML = yaml.safe_load(open(CONFIG_PATH, 'r'))
READ_CSV = pd.read_csv(DATA_READ_PATH).to_dict()

# informations
CHOOSE_IDX = np.random.randint(0, 51300)
ANCHOR_NUM = GENERATE_YAML['anchor_num']
ANCHOR_POS = [GENERATE_YAML[f'anchor{idx}'] for idx in range(ANCHOR_NUM)]
ANCHOR_READING = [READ_CSV[f'anchor{idx}'][CHOOSE_IDX] for idx in range(ANCHOR_NUM)]
TAG_POS = [READ_CSV[f'tag_x'][CHOOSE_IDX], READ_CSV[f'tag_y'][CHOOSE_IDX], READ_CSV[f'tag_z'][CHOOSE_IDX]]
TAG_POS_X_RANGE = GENERATE_YAML['tag']['x_range']
TAG_POS_Y_RANGE = GENERATE_YAML['tag']['y_range']
TAG_POS_Z_RANGE = GENERATE_YAML['tag']['z_range']

# printout the informations
print("Anchors' position   : ", ANCHOR_POS)
print("Anchors' reading    : ", ANCHOR_READING)
print(f"Tags' pos. interval :  {TAG_POS_X_RANGE}, {TAG_POS_Y_RANGE}, {TAG_POS_Z_RANGE}")

# repetitively used functions
def calcError(guess_pos: list, anchor_pos: list, reading: float) -> float:
    a = anchor_pos
    p = guess_pos
    r = reading
    return np.sqrt((a[0] - p[0])**2.0 + (a[1] - p[1])**2.0 + (a[2] - p[2])**2.0) - r

def calcObjective(guess_pos: list) -> float:
    res = 0.0
    for i in range(ANCHOR_NUM):
        res += calcError(guess_pos, ANCHOR_POS[i], ANCHOR_READING[i])**2.0
    return float(res)


Anchors' position   :  [[0.0, 0.0, 0.0], [8.45, 0.0, 0.0], [8.45, 9.73, 0.0], [0.0, 9.73, 0.0]]
Anchors' reading    :  [5.269259835101714, 6.569908307545969, 7.635564502089565, 6.525155470915509]
Tags' pos. interval :  [0.5, 8.1], [0.5, 9.5], [0.0, 0.1]


## **1. Grey-Wolf Optimizer**

The Grey-Wolf Optimizer (GWO for short) works by determining the idea of wolf hunting. It works as follows.

### **1.1. Determine the Initial Parameters**

Let's start by choosing the two parameters, the number of wolves and the number of iterations.

In [108]:
NUM_OF_WOLVES = 8
NUM_OF_ITERATIONS = 200

Wolves are scattered randomly inside the possible tag's position. From that, they start to smell their prey via the objective function.

In [109]:
wolves_pos = []
wolves_obj = []
for i in range(NUM_OF_WOLVES):
    wolves_pos.append([
        TAG_POS_X_RANGE[0] + np.random.rand()*TAG_POS_X_RANGE[1], 
        TAG_POS_Y_RANGE[0] + np.random.rand()*TAG_POS_Y_RANGE[1], 
        TAG_POS_Z_RANGE[0] + np.random.rand()*TAG_POS_Z_RANGE[1]
    ])
    wolves_obj.append(calcObjective(wolves_pos[i]))

print("Wolves' position: ", wolves_pos)
print("Wolves' smelling: ", wolves_obj)

Wolves' position:  [[1.249936344783598, 7.406322028332688, 0.024870592590485865], [4.169115944486588, 9.374743187230376, 0.06966468058969057], [6.826155158296, 3.762122947596718, 0.09155992861435935], [3.3863612473916533, 1.0358457382434143, 0.05193626109789571], [7.42017259665345, 5.92946385683336, 0.02358460124677597], [4.728266271304081, 9.24875242616236, 0.07755753982707136], [3.3344985061533117, 9.252236553400536, 0.01221974766359032], [2.0900529248862747, 9.615407083239562, 0.03665921243672165]]
Wolves' smelling:  [34.26895817097943, 55.49802314693291, 21.051680626757488, 18.702553959945647, 35.14607785757199, 55.96610695712759, 53.065634804904754, 66.74311041612752]


Now for the final step in initialization, we determine the best three wolves which closest to the prey, namely alpha, beta, and omega.

In [110]:
ordered_idx = sorted(range(NUM_OF_WOLVES), key=lambda i: wolves_obj[i])[:3]
idx_alpha = ordered_idx[0]
idx_beta = ordered_idx[1]
idx_omega = ordered_idx[2]

print(idx_alpha, idx_beta, idx_omega)

3 2 0


### **1.2. The Iterations**

Now, once the alpha, beta, and omega wolves are known, every wolves are starting to follow them, since the three smell their prey the best. To achieve this, we update each wolf one by one according to the algorithm's rule.

In [111]:
def updateWolf(a: float, current_wolf: list, alpha_wolf: list, beta_wolf: list, omega_wolf: list) -> list:
    current = np.array(current_wolf)
    alpha = np.array(alpha_wolf)
    beta = np.array(beta_wolf)
    omega = np.array(omega_wolf)

    Aval = a*(2.0*np.random.rand() - 1.0)
    Cval = 2.0*np.random.rand()
    Dval = np.abs(Cval*alpha - current)
    X1 = alpha - Aval*Dval

    Aval = a*(2.0*np.random.rand() - 1.0)
    Cval = 2.0*np.random.rand()
    Dval = np.abs(Cval*beta - current)
    X2 = beta - Aval*Dval

    Aval = a*(2.0*np.random.rand() - 1.0)
    Cval = 2.0*np.random.rand()
    Dval = np.abs(Cval*omega - current)
    X3 = omega - Aval*Dval

    Xnew = (X1 + X2 + X3)/3.0
    return Xnew.tolist() if calcObjective(Xnew.tolist()) < calcObjective(current_wolf) else current_wolf

for it in range(NUM_OF_ITERATIONS):
    a = 2.0*((it + 1)/NUM_OF_ITERATIONS)
    
    for i in range(NUM_OF_WOLVES):
        wolves_pos[i] = updateWolf(a, wolves_pos[i], wolves_pos[idx_alpha], wolves_pos[idx_beta], wolves_pos[idx_omega])
        wolves_obj[i] = calcObjective(wolves_pos[i])

    ordered_idx = sorted(range(NUM_OF_WOLVES), key=lambda i: wolves_obj[i])[:3]
    idx_alpha = ordered_idx[0]
    idx_beta = ordered_idx[1]
    idx_omega = ordered_idx[2]

print("Prey (tag) catched at: ", wolves_pos[idx_alpha])
print("Actual prey location : ", TAG_POS)
print("With objective of    : ", wolves_obj[idx_alpha])

Prey (tag) catched at:  [3.6775201062314817, 3.815287127329448, 0.053962235149060685]
Actual prey location :  [3.2999999999999994, 4.1, 0.0]
With objective of    :  0.40682959622035864
